In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from ipywidgets import interact, widgets, Button, VBox, HBox
import geopandas as gpd
from IPython.display import display

In [9]:
files = ['gdańsk_weather_data_final.csv', 'kraków_weather_data_final.csv', 'warszawa_weather_data_final.csv', 'wrocław_weather_data_final.csv', 'szczecin_weather_data_final.csv']
city_coordinates = {
    'Gdańsk': (54.3520, 18.6466),
    'Kraków': (50.0647, 19.9450),
    'Warszawa': (52.2297, 21.0122),
    'Wrocław': (51.1079, 17.0385),
    'Szczecin': (53.4285, 14.5528)
}

In [10]:
data_frames = []
for file in files:
    df = pd.read_csv(file)
    df['time'] = pd.to_datetime(df['time'])
    data_frames.append(df)

df = pd.concat(data_frames, ignore_index=True)

# Debug: Wyświetl pierwsze kilka wierszy wczytanych danych
print(df.head()) 

        time      tavg      prcp  wspd    pres    City  number
0 2024-01-01  1.900000 -0.162265  10.4  1004.6  Gdańsk       1
1 2024-01-02  1.500000  2.000000   9.9  1003.2  Gdańsk       2
2 2024-01-03 -4.344857  0.360638  19.5   992.8  Gdańsk       3
3 2024-01-04 -3.800000  7.600000  15.3   993.3  Gdańsk       4
4 2024-01-05 -5.700000  0.000000  12.9  1007.5  Gdańsk       5


In [11]:
# Wczytaj granice Polski za pomocą geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
poland = world[world.name == 'Poland']

# Globalna zmienna do przechowywania aktualnego wykresu
current_figure = None

# Funkcja do rysowania mapy
def plot_map(selected_date):
    global current_figure

    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([14, 25, 48, 56])  # Ustaw granice mapy dla Polski
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Dodanie granic Polski
    poland_boundary = poland.geometry.unary_union
    ax.add_geometries([poland_boundary], ccrs.PlateCarree(), facecolor='none', edgecolor='black')

    # Filtruj dane według wybranej daty
    filtered_data = df[df['time'].dt.date == selected_date]

    # Debug: Wyświetl filtrowane dane
    print(f"Selected date: {selected_date}")
    print(filtered_data)

    for idx, row in filtered_data.iterrows():
        city = row['City']
        lat, lon = city_coordinates[city]

        info = (f"{city}\n"
                f"Tavg: {row['tavg']}°C\n"
                f"Prcp: {row['prcp']} mm\n"
                f"Wspd: {row['wspd']} km/h\n"
                f"Pres: {row['pres']} hPa\n")

        plt.plot(lon, lat, marker='o', color='red', markersize=10, transform=ccrs.Geodetic())
        plt.text(lon + 0.5, lat - 0.5, info, transform=ccrs.Geodetic(), fontsize=8, bbox=dict(facecolor='white', alpha=0.5))

    plt.title(f"Weather Data for {selected_date.strftime('%Y-%m-%d')}")
    plt.show()

    current_figure = fig

# Funkcja do zapisywania wykresu
def save_current_figure(_):
    if current_figure:
        current_figure.savefig('weather_data_map.png')
        print("Wykres zapisany jako 'weather_data_map.png'")
    else:
        print("Brak wykresu do zapisania")

# Znajdź unikalne daty
unique_dates = df['time'].dt.date.unique()

# Debug: Wyświetl unikalne daty
print(unique_dates)

# Utwórz interaktywny suwak
date_slider = widgets.SelectionSlider(options=unique_dates, description="Date", continuous_update=False)
interact(plot_map, selected_date=date_slider)

# Utwórz przycisk do zapisywania wykresu
save_button = Button(description="Save Map")
save_button.on_click(save_current_figure)

# Wyświetl suwak i przycisk
display(VBox([date_slider, save_button]))

[datetime.date(2024, 1, 1) datetime.date(2024, 1, 2)
 datetime.date(2024, 1, 3) datetime.date(2024, 1, 4)
 datetime.date(2024, 1, 5) datetime.date(2024, 1, 6)
 datetime.date(2024, 1, 7) datetime.date(2024, 1, 8)
 datetime.date(2024, 1, 9) datetime.date(2024, 1, 10)
 datetime.date(2024, 1, 11) datetime.date(2024, 1, 12)
 datetime.date(2024, 1, 13) datetime.date(2024, 1, 14)
 datetime.date(2024, 1, 15) datetime.date(2024, 1, 16)
 datetime.date(2024, 1, 17) datetime.date(2024, 1, 18)
 datetime.date(2024, 1, 19) datetime.date(2024, 1, 20)
 datetime.date(2024, 1, 21) datetime.date(2024, 1, 22)
 datetime.date(2024, 1, 23) datetime.date(2024, 1, 24)
 datetime.date(2024, 1, 25) datetime.date(2024, 1, 26)
 datetime.date(2024, 1, 27) datetime.date(2024, 1, 28)
 datetime.date(2024, 1, 29) datetime.date(2024, 1, 30)
 datetime.date(2024, 1, 31) datetime.date(2024, 2, 1)
 datetime.date(2024, 2, 2) datetime.date(2024, 2, 3)
 datetime.date(2024, 2, 4) datetime.date(2024, 2, 5)
 datetime.date(2024, 2, 

C:\Users\ssiko\AppData\Local\Temp\ipykernel_17560\2176969051.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


interactive(children=(SelectionSlider(continuous_update=False, description='Date', options=(datetime.date(2024…

Wykres zapisany jako 'weather_data_map.png'
